In [ ]:
#-------------------------------------------------------------------------------------------#
#-------------------------------------------------------------------------------------------#
#-------------------------------------------------------------------------------------------#

#PUD_Hosp_Data_Script 
#Author: Aishwarya Rameshkumar, Cornell University
#Script to upload csv data into mysql table, add uniform noise to the dataset, setup the
#table to include functional dependencies

#[ Next Steps: Read PUD PAPER FULLY, Read MLN PAPER, TRAIN THE MLN MODEL ]


#Assumptions:
#1. Input data is in CSV form
#2. Input data is "ground truth" or assumed to be all correct 
#2. MySQL table will include additional preceding column for indexing "pd_index"

#-------------------------------------------------------------------------------------------#
#-------------------------------------------------------------------------------------------#
#-------------------------------------------------------------------------------------------#

import csv
import MySQLdb
import pandas
import mysql.connector
from sqlalchemy import *
import random

print 'Attempting to connect to database...'
#dbname_in = raw_input("Type in DB name: ") 
#user_in = raw_input("Type in DB username: ") 
pw_in = raw_input("Type in DB password: ")
#table_in = raw_input("Type in table name: ")
#Temporary hardcode
dbname_in = 'hosp_test' 
user_in = 'root' 
table_in = 'hosp_table_pyeng'

conn_string = 'mysql+mysqldb://{}:{}@localhost/{}'.format(user_in,pw_in,dbname_in)
engine = create_engine(conn_string, pool_pre_ping=True)
metadata = MetaData(bind=engine)

print 'Start'

hosp_dataframe = pandas.read_csv('Payment and Value of Care - Hospital.csv')

#Assumption: when the following line to create a mysql table, it adds a preceding column
#containing a unique index for each row of the original data table
#Also make sure to run 'SET GLOBAL max_allowed_packet=1073741824;' in mysql when starting new session
#or permanently change the value in the config file
#https://stackoverflow.com/questions/8062496/how-to-change-max-allowed-packet-size

#hosp_dataframe.to_sql(name = 'hosp_table_pyeng', index_label = 'pd_index', con=engine, if_exists = 'replace')


conn = MySQLdb.connect(host='localhost',
    user=user_in,
    passwd=pw_in,
    db=dbname_in) 

conn.query("""select column_name from information_schema.columns where table_schema='hosp_test' and table_name='{}';""".format(table_in))
r = conn.store_result()
col_names_tupl = r.fetch_row(maxrows=0, how=0)
conn.commit()
col_names = [x[0] for x in col_names_tupl]
print 'COLUMN NAMES'
print col_names

count = 0;
for val in col_names:
    conn.query("""select `{}` from `{}`""".format(val,table_in))
    r2 = conn.store_result()
    row_names_tupl = r2.fetch_row(maxrows=0, how=0)
    conn.commit()
    row_names = [(x[0]) for x in row_names_tupl] #leave values in original long format  
    """if count <5:
        print 'ROW NAMES'
        print row_names
    count = count + 1"""
print 'checkpoint'

conn.query("""select count(*) from `{}`;""".format(table_in))
r4 = conn.store_result()
num_rows_raw = r4.fetch_row(maxrows=0, how=0)
conn.commit()
num_rows = num_rows_raw[0][0]
#print num_rows
iter_rows = num_rows+1
threshold = 0.05

try:
    col_names.remove("pd_index") #STILL DEPENDENT ON THE TABLE - CHANGE!
except ValueError:
    pass

updated = 0;
for val in col_names:
    for x in range(1,iter_rows):
        rand_num = random.uniform(0,1)
        if (rand_num <= 0.0005):
            #print 'Random number generated is: ' + str(rand_num) 
            conn.query("""select `{}` from `{}` where `pd_index`= '{}';""".format(val, table_in, x))
            r5 = conn.store_result()
            cur_row_raw = r5.fetch_row(maxrows=0, how=0)
            conn.commit()
            if cur_row_raw:
                cur_row_val = cur_row_raw[0][0]
            #print 'Column is: ' + str(val)
            #print 'Current row value: ' + str(cur_row_val)
            rand_row_val = cur_row_val
            while (rand_row_val == cur_row_val):
                conn.query("""select `{}` from `{}` order by rand() limit 1;""".format(val, table_in))
                r6 = conn.store_result()
                rand_row_raw = r6.fetch_row(maxrows=0, how=0)
                conn.commit()
                rand_row_val = rand_row_raw[0][0]   
            #print 'Random row value: ' + str(rand_row_val)
            if isinstance(rand_row_val, str):
                rand_row_val = rand_row_val.replace("'","''")
            conn.query("""update `{}` set `{}` = '{}' where `pd_index` = '{}';""".format(table_in, val, rand_row_val, x))
            conn.commit()
            updated = updated + 1
            #print 'Cell updated ' + str(updated) 
            
            
conn.close()
print 'Total number of updates is: ' + str(updated)
print 'done'

